In [0]:
import sys
sys.path.append('../')

In [0]:
import requests 
import pandas as pd
import urllib.parse
import json
from datetime import datetime
import pyspark.sql.types as T
from common import *

In [0]:
dbutils.widgets.text("datestr", "", "Date String")
datestr = dbutils.widgets.get("datestr")
print(datestr)

#if not passed as part of load - create ur own 
if datestr == '':
    today = datetime.now()
    datestr = today.strftime("%Y%m%d_%H%M%S")

In [0]:
# token=dbutils.secrets.get()

In [0]:
##Get all company properties (custom and default) and load into string used in URL
url= f'https://api.hubapi.com/properties/v1/companies/properties'
headers = {
  'content-type': 'application/json',
  'authorization': f'Bearer {token}'
}
r=requests.request('GET',url=url,headers=headers)
properties=r.json()

property_string=''
for i in range(0,len(properties)):
    p = properties[i].get('name')
    property_string +=f'&properties={p}'

In [0]:
#Get all companies
company_list = []
url = "https://api.hubapi.com/companies/v2/companies/paged?" 
parameter_dict = {'limit': 250, 'offset': 0}
headers = {
  'content-type': 'application/json',
  'authorization': f'Bearer {token}'
}

file_count = 0
# Paginate your request using offset
has_more = True
while has_more:
    
    parameters = urllib.parse.urlencode(parameter_dict)
    get_url = url + parameters + property_string
    
    try:
        r = request_with_retry(get_url, headers)
        
    except Exception as e: 
        if len(company_list)>0:
            file_name=str(file_count).zfill(5)
            save_new_json(company_list, 'companies', datestr, file_name)
            file_count += 1
            company_list = []
    
    response_dict = json.loads(r.text)       
    company_list.extend(response_dict['companies'])
    has_more=response_dict['has-more']
    parameter_dict['offset'] = response_dict['offset']
    
    if len(company_list) > 10000:
        file_name = str(file_count).zfill(5)
        save_new_json(company_list, 'companies', datestr, file_name)
        file_count += 1
        company_list = []
    if not has_more:
        file_name = str(file_count).zfill(5)
        save_new_json(company_list, 'companies', datestr, file_name)
        break 
